## Driving a Taxi in Buenos Aires
### Part 1 :The Problem

My friend Juan who lives in the suburbs lost his job and is thinking to drive a taxi to earn some income until he can get another job. Fortunately for him the taxi owner allows him to choose his job schedule. He would also like to move to a neighborhood with high taxi demand to minimize driving but good gastronomy options as he is lazy to cook his own meals.  
  So in this assignment I will be answering two questions:  
#### What are the most profitable days and shifts (morning, evenings or nights) to drive a taxi?  
#### What is the neighborhood with the best balance of taxi traffic and restaurants?



### The Data

I will be using for this the Buenos Aires Government open Portal that provides the data of taxi trips and Foursquare to rate the neighborhoods:  
[data.buenosaires.gob.ar](https://data.buenosaires.gob.ar/dataset)  
FourSquare API from Course labs

In [1]:
# Instal wget to download files
!pip install wget

# Instal geomery engine
!pip install shapely

# Instal map engine
!pip install folium

# Instal beautifulsoup module
!pip install beautifulsoup4

!conda install -c conda-forge geopy --yes # Foursquare API 

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9681 sha256=eaae5537f358e190a5b42fc53aa1fe66cf6ec8764daa5368954814e164bf14ac
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
     |████████████████████████████████| 1.0 MB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 4.2 MB/s  eta 0:00:01
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: | 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                               /failed

UnsatisfiableError: The following specifications were 

#### Import Buenos Aires Neighborhoods from the Buenos AIres Government Site 

In [2]:
# Import necessary libraries
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import numpy as np
import requests
from shapely.geometry import Point, Polygon, box, shape
from shapely import wkt
from datetime import datetime as dt 

In [3]:
!wget -q -O 'BA_data.json' http://cdn.buenosaires.gob.ar/datosabiertos/datasets/barrios/barrios.geojson
print('Data downloaded!')

Data downloaded!


#### Load and explore the data

In [4]:
with open('BA_data.json') as json_data:
    BA_data = json.load(json_data)

Let's take a quick look at the data.

In [5]:
BA_data

{'type': 'FeatureCollection',
 'name': 'barrios_badata',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'barrio': 'CHACARITA',
    'comuna': 15,
    'perimetro': 7724.85295457,
    'area': 3115707.10627},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-58.4528200492791, -34.5959886570639],
      [-58.453655193137, -34.5965557163041],
      [-58.4537674321647, -34.5966342484152],
      [-58.4538163134148, -34.5966684788922],
      [-58.4547947928051, -34.5973527273644],
      [-58.4554840815948, -34.5978347610252],
      [-58.4559204833296, -34.5976953435829],
      [-58.4560093721285, -34.5976669530232],
      [-58.4560576047802, -34.5976515472868],
      [-58.4562363723257, -34.5975900748435],
      [-58.4564940053845, -34.597501563774],
      [-58.4570733158433, -34.5973024999733],
      [-58.4576142986162, -34.5971165306991],
      [-58.4578298572666, -34.5970394940224],
      [-58.45

I will save the name of the neighborhoods in data frame using the 'features' key

In [6]:
neighborhoods_data = BA_data['features']

In [7]:
# Let's check one item
neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'barrio': 'CHACARITA',
  'comuna': 15,
  'perimetro': 7724.85295457,
  'area': 3115707.10627},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-58.4528200492791, -34.5959886570639],
    [-58.453655193137, -34.5965557163041],
    [-58.4537674321647, -34.5966342484152],
    [-58.4538163134148, -34.5966684788922],
    [-58.4547947928051, -34.5973527273644],
    [-58.4554840815948, -34.5978347610252],
    [-58.4559204833296, -34.5976953435829],
    [-58.4560093721285, -34.5976669530232],
    [-58.4560576047802, -34.5976515472868],
    [-58.4562363723257, -34.5975900748435],
    [-58.4564940053845, -34.597501563774],
    [-58.4570733158433, -34.5973024999733],
    [-58.4576142986162, -34.5971165306991],
    [-58.4578298572666, -34.5970394940224],
    [-58.4579314199407, -34.5970031094655],
    [-58.4579383597537, -34.5970006235554],
    [-58.457939151188, -34.5970004324802],
    [-58.4580314840712, -34.5969781271249],
    [-58.4582467474022, -34.5969

The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

As the neighborhoods are given in the form of polygons, we can shapey  to find the centroid of each and pass it to a numpy array

Now we can fill the dataframe

In [9]:
for data in neighborhoods_data:
    borough = data['properties']['comuna'] 
    neighborhood_name = data['properties']['barrio']
    neighborhood_latlon = np.array(shape(data['geometry']).centroid)
    
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,15,CHACARITA,-34.588369,-58.454180
1,15,PATERNAL,-34.597421,-58.468665
2,15,VILLA CRESPO,-34.598830,-58.442725
3,11,VILLA DEL PARQUE,-34.604247,-58.490677
4,5,ALMAGRO,-34.609227,-58.421745


Let´s see how many Borughs and Neighborhoods are in Buenos Aires

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 48 neighborhoods.


### Let´s see a map

In [12]:
import folium

latitude = neighborhoods.Latitude.mean()
longitude = neighborhoods.Longitude.mean()
map = folium.Figure(width=1000, height=500)
map = folium.Map(location=[latitude, longitude], zoom_start=12,).add_to(map)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)
map

#### Taxi traffic dataset

In [13]:
# Let´s download the file with taxi trips and explore it
BATaxi = pd.read_csv('https://query.data.world/s/otrzxnb4wfdakh7ezdtgvv4iqnjsoj', decimal=',',  parse_dates=[2,3])
BATaxi.head()

,id_viaje_r,id_taxista_r,fecha_inicio,fecha_fin,duracion,origen_viaje_x,origen_viaje_y,destino_viaje_x,destino_viaje_y,cantidad_pasajeros
0,1,499,2017-05-01 00:12:48,2017-05-01 00:31:09,1101,-58.387303,-34.594316,-58.431452,-34.578193,3
1,2,715,2017-05-01 00:17:21,2017-05-01 00:28:36,675,-58.413992,-34.582760,-58.438494,-34.587420,1
2,3,429,2017-05-01 00:16:03,2017-05-01 00:36:33,1230,-58.429912,-34.599420,-58.483171,-34.564596,1
3,4,164,2017-05-01 00:34:39,2017-05-01 00:54:57,1218,-58.447841,-34.633739,-58.439394,-34.593648,2
4,5,499,2017-05-01 00:32:02,2017-05-01 00:48:37,995,-58.433165,-34.579621,-58.403720,-34.599863,2


In [14]:
#Let´s change the titles to english and use the TripID as index. Let´s see how many entries are there 
BATaxi.columns = ['TripID','DriverID','Start_Date','End_Date','Trip_Time(s)','trip_ori_x','trip_ori_y','trip_dest_x','trip_dest_y','num_pass']

print('The dataframe has {} taxi trips recorded.'.format(
        len(BATaxi['TripID'].unique())))

BATaxi.set_index('TripID', inplace=True) 

BATaxi.head()

The dataframe has 19148 taxi trips recorded.


,DriverID,Start_Date,End_Date,Trip_Time(s),trip_ori_x,trip_ori_y,trip_dest_x,trip_dest_y,num_pass
TripID,,,,,,,,,
1,499,2017-05-01 00:12:48,2017-05-01 00:31:09,1101,-58.387303,-34.594316,-58.431452,-34.578193,3
2,715,2017-05-01 00:17:21,2017-05-01 00:28:36,675,-58.413992,-34.582760,-58.438494,-34.587420,1
3,429,2017-05-01 00:16:03,2017-05-01 00:36:33,1230,-58.429912,-34.599420,-58.483171,-34.564596,1
4,164,2017-05-01 00:34:39,2017-05-01 00:54:57,1218,-58.447841,-34.633739,-58.439394,-34.593648,2
5,499,2017-05-01 00:32:02,2017-05-01 00:48:37,995,-58.433165,-34.579621,-58.403720,-34.599863,2


### Part 2: Exploring the Data

#### We can now add the time of the date for each trip to analyze the workload by day period

In [15]:
# We create a function to assign the work shifts according to the trip start

def f(x):
    if (x > 3) and (x <= 12):
        return 'Morning'
    elif (x > 12) and (x <= 19):
        return'AfterNoon'
    elif (x > 19) and (x <= 24):
        return'Night'
    elif (x <= 3):
        return'Night'

# And apply it to the dataframe
   
BATaxi['Work_Shift'] = BATaxi.Start_Date.dt.hour.apply(f)

BATaxi.head()

,DriverID,Start_Date,End_Date,Trip_Time(s),trip_ori_x,trip_ori_y,trip_dest_x,trip_dest_y,num_pass,Work_Shift
TripID,,,,,,,,,,
1,499,2017-05-01 00:12:48,2017-05-01 00:31:09,1101,-58.387303,-34.594316,-58.431452,-34.578193,3,Night
2,715,2017-05-01 00:17:21,2017-05-01 00:28:36,675,-58.413992,-34.582760,-58.438494,-34.587420,1,Night
3,429,2017-05-01 00:16:03,2017-05-01 00:36:33,1230,-58.429912,-34.599420,-58.483171,-34.564596,1,Night
4,164,2017-05-01 00:34:39,2017-05-01 00:54:57,1218,-58.447841,-34.633739,-58.439394,-34.593648,2,Night
5,499,2017-05-01 00:32:02,2017-05-01 00:48:37,995,-58.433165,-34.579621,-58.403720,-34.599863,2,Night


#### Let´s assign the origin and destiny neighborhood to each taxi trip 

In [16]:
# Function to get neighborhood with lat & long as inputs

def get_neighb(lon, lat):
    point = Point(lon, lat)
    for feature in neighborhoods_data:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
            return(feature['properties']['barrio'])
    

# Initialize list
tripo = tripd = ['']*BATaxi.shape[0]

for i in range(len(BATaxi)):
    tripo[i] = get_neighb(BATaxi.iloc[i,4], BATaxi.iloc[i,5])
    tripd[i] = get_neighb(BATaxi.iloc[i,6], BATaxi.iloc[i,7])
    
# create new dataframe to store both origin and destinations lists
BATaxi_neighb= pd.DataFrame(list(zip(tripo,tripd)),columns=['Origin','Destination'])

# Clean Datframe and merge to add neighborhood to each trip latitude/longitude
BATAxi = BATaxi. drop(['DriverID','Start_Date','End_Date','trip_ori_x','trip_ori_y','trip_dest_x','trip_dest_y'],axis=1)
BATaxi_neighb = BATAxi.join(BATaxi_neighb)

#let´s see how it looks like
BATaxi_neighb.head()

,Trip_Time(s),num_pass,Work_Shift,Origin,Destination
TripID,,,,,
1,1101,3,Night,PALERMO,PALERMO
2,675,1,Night,VILLA URQUIZA,VILLA URQUIZA
3,1230,1,Night,VILLA CRESPO,VILLA CRESPO
4,1218,2,Night,BALVANERA,BALVANERA
5,995,2,Night,CHACARITA,CHACARITA


#### We can check which are the Neighborhoods and work shifts with the most trips

In [18]:
# we can use a pivot table to group items and sort results

Trips = BATaxi_neighb.pivot_table(index=['Origin','Work_Shift'],aggfunc=['count','sum'],values=('Trip_Time(s)'))
Trips.columns=['Trips_Count','Trips_Lenght']
Trips = Trips.reset_index().sort_values(['Trips_Count','Trips_Lenght'], ascending=[0,0]).set_index(['Origin','Work_Shift'])
Trips['Length_by_Trip(Min)'] = (Trips['Trips_Lenght']/Trips['Trips_Count']/60).round(2)
Trips.head(10) 

Trips_Count  Trips_Lenght  Length_by_Trip(Min)
Origin      Work_Shift                                                
PALERMO     Night              1478       1341572                15.13
            Morning             908        887112                16.28
            AfterNoon           750        692934                15.40
RECOLETA    Night               711        649310                15.22
CABALLITO   Night               683        625038                15.25
ALMAGRO     Night               474        426401                14.99
RECOLETA    Morning             461        438785                15.86
BALVANERA   Night               454        412540                15.14
SAN NICOLAS Morning             391        371300                15.83
BALVANERA   Morning             386        356934                15.41

### Part 3: Conclusion and Neighborhood Exploration

##### Palermo is the neighborhoods with most taxi traffic by a large account. We can also note that the night shift is also the most profitable while the trip lenght in consistently around 15 minutes across shifts and Neighborhoods 


##### We will now explore the neighborhoods for our friend Juan using Foursquare to find neighborhoods with good variety of gastronomy to live in


In [74]:
# The code was removed by Watson Studio for sharing.

In [75]:
#We can leverage the function seen in the lab to get venues

def getNearbyVenues(names, latitudes, longitudes, radius=800):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
# let's run the query function to our neighborhoods dataframe.

neigh_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

CHACARITA
PATERNAL
VILLA CRESPO
VILLA DEL PARQUE
ALMAGRO
CABALLITO
VILLA SANTA RITA
MONTE CASTRO
VILLA REAL
FLORES
FLORESTA
CONSTITUCION
SAN CRISTOBAL
BOEDO
VELEZ SARSFIELD
VILLA LURO
PARQUE PATRICIOS
MATADEROS
VILLA LUGANO
SAN TELMO
SAAVEDRA
COGHLAN
VILLA URQUIZA
COLEGIALES
BALVANERA
VILLA GRAL. MITRE
PARQUE CHAS
AGRONOMIA
VILLA ORTUZAR
BARRACAS
PARQUE AVELLANEDA
PARQUE CHACABUCO
NUEVA POMPEYA
PALERMO
VILLA RIACHUELO
VILLA SOLDATI
VILLA PUEYRREDON
VILLA DEVOTO
LINIERS
VERSALLES
PUERTO MADERO
MONSERRAT
SAN NICOLAS
BELGRANO
RECOLETA
RETIRO
NUÑEZ
BOCA


#### Let's check the size of the resulting dataframe

In [90]:
print(neigh_venues.shape)
neigh_venues.head()

(2022, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,CHACARITA,-34.588369,-58.45418,El Imperio de la Pizza,-34.586890,-58.454967,Pizza Place
1,CHACARITA,-34.588369,-58.45418,Albamonte Ristorante,-34.587803,-58.453075,Argentinian Restaurant
2,CHACARITA,-34.588369,-58.45418,Santos 4040,-34.588822,-58.449863,Theater
3,CHACARITA,-34.588369,-58.45418,Sede Central de Whisky,-34.586750,-58.450596,Whisky Bar
4,CHACARITA,-34.588369,-58.45418,Fábrica de Churros Olleros,-34.586983,-58.453640,Bakery


#### Let's find out how many unique categories can be curated from all the returned venues

In [84]:
print('There are {} uniques categories.'.format(len(neigh_venues['Venue Category'].unique())))

There are 230 uniques categories.


In [107]:
# one hot encoding for the Venue Categories
BA_onehot = pd.get_dummies(neigh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
BA_onehot['Neighborhood'] = neigh_venues['Neighborhood']

# move neighborhood column to the first column

fixed_columns = [BA_onehot.columns[-1]] + list(BA_onehot.columns[:-1])
BA_onehot = BA_onehot[fixed_columns]

BA_onehot.head()

,Women's Store,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,...,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Let´s see the new dataframe size 

In [103]:
BA_onehot.shape

(2022, 230)

#### We can now see the mean frequency of each category in each Neighborhood

In [106]:
neigh_grouped = BA_onehot.groupby('Neighborhood').mean().reset_index()
neigh_grouped

,Neighborhood,Women's Store,Accessories Store,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,...,Used Bookstore,Vacation Rental,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop
0,AGRONOMIA,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
1,ALMAGRO,0.000000,0.00000,0.000000,0.000000,0.115385,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.025641,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
2,BALVANERA,0.000000,0.00000,0.000000,0.000000,0.015873,0.000000,0.000000,0.031746,0.000000,...,0.00,0.00,0.000000,0.000000,0.015873,0.000000,0.00,0.000000,0.00,0.000000
3,BARRACAS,0.000000,0.00000,0.000000,0.000000,0.176471,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
4,BELGRANO,0.000000,0.00000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.010000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
5,BOCA,0.000000,0.00000,0.000000,0.000000,0.052632,0.052632,0.000000,0.052632,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
6,BOEDO,0.000000,0.00000,0.000000,0.000000,0.263158,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000
7,CABALLITO,0.000000,0.00000,0.000000,0.000000,0.036145,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.012048,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.012048
8,CHACARITA,0.000000,0.00000,0.000000,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.021739,0.021739,0.000000,0.000000,0.00,0.021739,0.00,0.000000
9,COGHLAN,0.000000,0.00000,0.000000,0.000000,0.108108,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000


#### Let's use the function to sort the venues in descending order.

In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [111]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neigh_grouped['Neighborhood']

for ind in np.arange(neigh_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neigh_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AGRONOMIA,Soccer Field,Pizza Place,Gym / Fitness Center,Bus Stop,Café,Farmers Market,Sports Club,Steakhouse,Garden Center,Restaurant
1,ALMAGRO,Argentinian Restaurant,Ice Cream Shop,Pizza Place,Café,Bar,Italian Restaurant,Indie Theater,Hotel,Gym,Dessert Shop
2,BALVANERA,Café,Pizza Place,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Bakery,Theater,Peruvian Restaurant,Electronics Store,BBQ Joint
3,BARRACAS,Argentinian Restaurant,Plaza,Athletics & Sports,Sporting Goods Shop,General Entertainment,Clothing Store,Restaurant,Soccer Field,Gym,Electronics Store
4,BELGRANO,Café,BBQ Joint,Pizza Place,Chinese Restaurant,Coffee Shop,Italian Restaurant,Grocery Store,Tea Room,Sushi Restaurant,Restaurant


#### Let´s see which are the ones for Palermo

In [113]:
neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighborhood'] == 'PALERMO']

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,PALERMO,Argentinian Restaurant,Ice Cream Shop,Sushi Restaurant,Nightclub,Supermarket,Burger Joint,Japanese Restaurant,Deli / Bodega,Coffee Shop,Cocktail Bar


#### Seems that Palermo is the right choice! It has plenty of restaurants and a few Nightclubs. That probably explains the high taxi traffic in the night shift.